In [ ]:
# !pip install numpy pandas xarray xskillscore
# !pip install cdsapi
# !pip install matplotlib cartopy
# !pip install cfgrib

# **1.LOAD THE REQUIRED LIBRARIES**

In [ ]:
# CDS API
import cdsapi

# Libraries for working with multi-dimensional arrays
import xarray as xr
import pandas as pd
import numpy as np

# Forecast verification metrics with xarray
import xskillscore as xs

# Date and calendar libraries
from dateutil.relativedelta import relativedelta
import calendar

# Libraries for plotting and geospatial data visualisation
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Disable warnings for data download via API and matplotlib (do I need both???)
import warnings
warnings.filterwarnings('ignore')

# **2.READ THE DATA**

In [ ]:
DATAIN ="/kaggle/input/seasonal-forcast-data/DATASETS"
DATAOUT="/kaggle/working/"
config = dict(
    list_vars = ['2m_temperature', ],
    hcstarty = 1993,
    hcendy = 2016,
    start_month = 11,
)

In [ ]:
import os

SCOREDIR = DATAOUT + '/SF/scores'
PLOTSDIR = DATAOUT + f'/SF/plots/stmonth{config["start_month"]:02d}'

for directory in [DATAIN, SCOREDIR, PLOTSDIR]:
    # Check if the directory exists
    if not os.path.exists(directory):
        # If it doesn't exist, create it
        os.makedirs(directory)
        print(f'Creating folder {directory}')

In [ ]:
# For the re-shaping of time coordinates in xarray.Dataset we need to select the right one
#  -> burst mode ensembles (e.g. ECMWF SEAS5) use "time". This is the default option in this notebook
#  -> lagged start ensembles (e.g. MetOffice GloSea6) use "indexing_time" (see CDS documentation about nominal start date)
st_dim_name = 'time' if not config.get('isLagged',False) else 'indexing_time'

In [ ]:
print('Reading HCST data from file')
hcst_fname=DATAIN + '/ecmwf_51_1993-2016_monthly_mean_11_234_45_-30_-2.5_60.grib'
hcst = xr.open_dataset(hcst_fname,engine='cfgrib', backend_kwargs=dict(time_dims=('forecastMonth', st_dim_name),filter_by_keys={'paramId':167}))
hcst = hcst.chunk({'forecastMonth':1, 'latitude':'auto', 'longitude':'auto'})  #force dask.array using chunks on leadtime, latitude and longitude coordinate
hcst = hcst.rename({'latitude':'lat','longitude':'lon', st_dim_name:'start_date'})

# **2.a : ANOMALIE**

In [ ]:
hcst

In [ ]:
print ('Re-arranging time metadata in xr.Dataset object')
# Add start_month to the xr.Dataset
start_month = pd.to_datetime(hcst.start_date.values[0]).month
hcst = hcst.assign_coords({'start_month':start_month})
# Add valid_time to the xr.Dataset
vt = xr.DataArray(dims=('start_date','forecastMonth'), coords={'forecastMonth':hcst.forecastMonth,'start_date':hcst.start_date})
vt.data = [[pd.to_datetime(std)+relativedelta(months=fcmonth-1) for fcmonth in vt.forecastMonth.values] for std in vt.start_date.values]
hcst = hcst.assign_coords(valid_time=vt)

# CALCULATE 3-month AGGREGATIONS
# NOTE rolling() assigns the label to the end of the N month period, so the first N-1 elements have NaN and can be dropped
print('Computing 3-month aggregation')
hcst_3m = hcst.rolling(forecastMonth=3).mean()
hcst_3m = hcst_3m.where(hcst_3m.forecastMonth>=3,drop=True)


# CALCULATE ANOMALIES (and save to file)
print('Computing anomalies 1m')
hcmean = hcst.mean(['number','start_date'])
anom = hcst - hcmean
anom = anom.assign_attrs(reference_period='{hcstarty}-{hcendy}'.format(**config))

print('Computing anomalies 3m')
hcmean_3m = hcst_3m.mean(['number','start_date'])
anom_3m = hcst_3m - hcmean_3m
anom_3m = anom_3m.assign_attrs(reference_period='{hcstarty}-{hcendy}'.format(**config))

print('Saving anomalies 1m/3m to netCDF files')
hcst_bname="ecmwf_51_1993-2016_monthly_mean"
anom.to_netcdf(f'{DATAOUT}/{hcst_bname}.1m.anom.nc')
anom_3m.to_netcdf(f'{DATAOUT}/{hcst_bname}.3m.anom.nc')

# **2.b : PROBABILITIES FOR TERCILE CATEGORIES**

In [ ]:
# We define a function to calculate the boundaries of forecast categories defined by quantiles
def get_thresh(icat,quantiles,xrds,dims=['number','start_date']):

    if not all(elem in xrds.dims for elem in dims):
        raise Exception('Some of the dimensions in {} is not present in the xr.Dataset {}'.format(dims,xrds))
    else:
        if icat == 0:
            xrds_lo = -np.inf
            xrds_hi = xrds.quantile(quantiles[icat],dim=dims)

        elif icat == len(quantiles):
            xrds_lo = xrds.quantile(quantiles[icat-1],dim=dims)
            xrds_hi = np.inf

        else:
            xrds_lo = xrds.quantile(quantiles[icat-1],dim=dims)
            xrds_hi = xrds.quantile(quantiles[icat],dim=dims)

    return xrds_lo,xrds_hi

In [ ]:
#  CALCULATE PROBABILITIES for tercile categories by counting members within each category

print('Computing probabilities (tercile categories)')
quantiles = [1/3., 2/3.]
numcategories = len(quantiles)+1

for aggr,h in [("1m",hcst), ("3m",hcst_3m)]:
    print(f'Computing tercile probabilities {aggr}')

    l_probs_hcst=list()
    for icat in range(numcategories):
        print(f'category={icat}')
        h_lo,h_hi = get_thresh(icat, quantiles, h)
        probh = np.logical_and(h>h_lo, h<=h_hi).sum('number')/float(h.dims['number'])
        # Instead of using the coordinate 'quantile' coming from the hindcast xr.Dataset
        # we will create a new coordinate called 'category'
        if 'quantile' in probh:
            probh = probh.drop('quantile')
        l_probs_hcst.append(probh.assign_coords({'category':icat}))

    print(f'Concatenating {aggr} tercile probs categories')
    probs = xr.concat(l_probs_hcst,dim='category')
    print(f'Saving {aggr} tercile probs netCDF files')
    probs.to_netcdf(f'{DATAOUT}/{hcst_bname}.{aggr}.tercile_probs.nc')

# **3 : COMPUTE SCORES**

## *3.1 DETERMINISTIC SCORES*

### *3.1.1 Read the Observation Data*

in this case we'll juste read the temperature data, because the precipitation data doesn't have the same time variable

In [ ]:
obs_fname=DATAIN+'/era5_monthly_stmonth{_NOVEMBER_}_{1993}-{2016}.grib'
era5_1deg = xr.open_dataset(obs_fname, engine='cfgrib',backend_kwargs={'filter_by_keys':{'paramId':167}})

In [ ]:
era5_1deg.valid_time.values

In [ ]:
# Renaming to match hindcast names
era5_1deg = era5_1deg.rename({'latitude':'lat','longitude':'lon','time':'start_date'}).swap_dims({'start_date':'valid_time'})




# Assign 'forecastMonth' coordinate values
fcmonths = [mm+1 if mm>=0 else mm+13 for mm in [t.month - config['start_month'] for t in pd.to_datetime(era5_1deg.valid_time.values)] ]
era5_1deg = era5_1deg.assign_coords(forecastMonth=('valid_time',fcmonths))
# Drop obs values not needed (earlier than first start date) - this is useful to create well shaped 3-month aggregations from obs.
era5_1deg = era5_1deg.where(era5_1deg.valid_time>=np.datetime64('{hcstarty}-{start_month:02d}-01'.format(**config)),drop=True)

# CALCULATE 3-month AGGREGATIONS
# NOTE rolling() assigns the label to the end of the N month period
print('Calculate observation 3-monthly aggregations')
# NOTE care should be taken with the data available in the "obs" xr.Dataset so the rolling mean (over valid_time) is meaningful
era5_1deg_3m = era5_1deg.rolling(valid_time=3).mean()
era5_1deg_3m = era5_1deg_3m.where(era5_1deg_3m.forecastMonth>=3)

# As we don't need it anymore at this stage, we can safely remove
# 'forecastMonth'
era5_1deg = era5_1deg.drop('forecastMonth')
era5_1deg_3m = era5_1deg_3m.drop('forecastMonth')

### *3.1.2 : Compute Deterministic Scores*

## *a- correlation*

In [ ]:
from os.path import join

# Loop over aggregations
anom_name = "ecmwf_51_1993-2016_monthly_mean"
for aggr in ['1m', '3m']:

    if aggr == '1m':
        o = era5_1deg
    elif aggr == '3m':
        o = era5_1deg_3m
    else:
        raise BaseException(f'Unknown aggregation {aggr}')

    print(f'Computing deterministic scores for {aggr}-aggregation')

    # Read anomalies file
    h = xr.open_dataset(f'{DATAOUT}/{hcst_bname}.{aggr}.anom.nc')
    is_fullensemble = 'number' in h.dims

    l_corr = list()
    l_corr_pval = list()

    for this_fcmonth in h.forecastMonth.values:
        print(f'forecastMonth={this_fcmonth}')
        thishcst = h.sel(forecastMonth=this_fcmonth).swap_dims({'start_date': 'valid_time'})
        thisobs = o.where(o.valid_time == thishcst.valid_time, drop=True)

        # Align the forecast and observation data along all common dimensions
        thishcst_em, thisobs_aligned = xr.align(thishcst, thisobs, join='inner')

        # If it's a full ensemble, take the mean over the 'number' dimension
        thishcst_em = thishcst_em if not is_fullensemble else thishcst_em.mean('number')

        l_corr.append(xs.spearman_r(thishcst_em, thisobs_aligned, dim='valid_time'))
        l_corr_pval.append(xs.spearman_r_p_value(thishcst_em, thisobs_aligned, dim='valid_time'))

    print(f'Concatenating (by fcmonth) correlation for {aggr}-aggregation')
    corr = xr.concat(l_corr, dim='forecastMonth')
    corr_pval = xr.concat(l_corr_pval, dim='forecastMonth')

    print(f'Saving to netCDF file correlation for {aggr}-aggregation')
    corr.to_netcdf(f'{DATAOUT}/scores/{hcst_bname}.{aggr}.corr.nc')
    corr_pval.to_netcdf(f'{DATAOUT}/scores/{hcst_bname}.{aggr}.corr_pval.nc')

## *b-root mean square error*

In [ ]:
from os.path import join

# Loop over aggregations
anom_name = "ecmwf_51_1993-2016_monthly_mean"
for aggr in ['1m', '3m']:

    if aggr == '1m':
        o = era5_1deg
    elif aggr == '3m':
        o = era5_1deg_3m
    else:
        raise BaseException(f'Unknown aggregation {aggr}')

    print(f'Computing deterministic scores for {aggr}-aggregation')

    # Read anomalies file
    h = xr.open_dataset(f'{DATAOUT}/{hcst_bname}.{aggr}.anom.nc')
    is_fullensemble = 'number' in h.dims

    l_rmse = list()

    for this_fcmonth in h.forecastMonth.values:
        print(f'forecastMonth={this_fcmonth}')
        thishcst = h.sel(forecastMonth=this_fcmonth).swap_dims({'start_date': 'valid_time'})
        thisobs = o.where(o.valid_time == thishcst.valid_time, drop=True)

        # Align the forecast and observation data along all common dimensions
        thishcst_em, thisobs_aligned = xr.align(thishcst, thisobs, join='inner')

        # If it's a full ensemble, take the mean over the 'number' dimension
        thishcst_em = thishcst_em if not is_fullensemble else thishcst_em.mean('number')

        l_rmse.append(xs.rmse(thishcst_em, thisobs_aligned, dim='valid_time'))

    print(f'Concatenating (by fcmonth) correlation for {aggr}-aggregation')
    rmse = xr.concat(l_rmse, dim='forecastMonth')

    print(f'Saving to netCDF file correlation for {aggr}-aggregation')
    rmse.to_netcdf(f'{DATAOUT}/scores/{hcst_bname}.{aggr}.rmse.nc')

## *c-R2*

In [ ]:
from os.path import join

# Loop over aggregations
anom_name = "ecmwf_51_1993-2016_monthly_mean"
for aggr in ['1m', '3m']:

    if aggr == '1m':
        o = era5_1deg
    elif aggr == '3m':
        o = era5_1deg_3m
    else:
        raise BaseException(f'Unknown aggregation {aggr}')

    print(f'Computing deterministic scores for {aggr}-aggregation')

    # Read anomalies file
    h = xr.open_dataset(f'{DATAOUT}/{hcst_bname}.{aggr}.anom.nc')
    is_fullensemble = 'number' in h.dims

    l_rsquared = list()

    for this_fcmonth in h.forecastMonth.values:
        print(f'forecastMonth={this_fcmonth}')
        thishcst = h.sel(forecastMonth=this_fcmonth).swap_dims({'start_date': 'valid_time'})
        thisobs = o.where(o.valid_time == thishcst.valid_time, drop=True)

        # Align the forecast and observation data along all common dimensions
        thishcst_em, thisobs_aligned = xr.align(thishcst, thisobs, join='inner')

        # If it's a full ensemble, take the mean over the 'number' dimension
        thishcst_em = thishcst_em if not is_fullensemble else thishcst_em.mean('number')

        l_rsquared.append(xs.r2(thishcst_em, thisobs_aligned, dim='valid_time'))

    print(f'Concatenating (by fcmonth) correlation for {aggr}-aggregation')
    rsquared = xr.concat(l_rsquared, dim='forecastMonth')

    print(f'Saving to netCDF file correlation for {aggr}-aggregation')
    rsquared.to_netcdf(f'{DATAOUT}/scores/{hcst_bname}.{aggr}.rsquared.nc')

### *3.1.2 : Compute probabilistic scores for tercile categories*

In [ ]:
from os.path import join

# Loop over aggregations
anom_name = "ecmwf_51_1993-2016_monthly_mean"
for aggr in ['1m', '3m']:

    if aggr == '1m':
        o = era5_1deg
    elif aggr == '3m':
        o = era5_1deg_3m
    else:
        raise BaseException(f'Unknown aggregation {aggr}')

    print(f'Computing deterministic scores for {aggr}-aggregation')

    # Read hindcast probabilities file
    probs_hcst = xr.open_dataset(f'{DATAOUT}/{hcst_bname}.{aggr}.tercile_probs.nc')

    l_roc = list()
    l_rps = list()
    l_rocss = list()
    l_bs = list()

    for this_fcmonth in probs_hcst.forecastMonth.values:
        print(f'forecastMonth={this_fcmonth}')
        thishcst = probs_hcst.sel(forecastMonth=this_fcmonth).swap_dims({'start_date': 'valid_time'})

        # CALCULATE probabilities from observations
        print('We need to calculate probabilities (tercile categories) from observations')
        l_probs_obs = list()

        # Align both forecast and observation data on 'valid_time'
        thiso = o.where(o.valid_time == thishcst.valid_time, drop=True)
        thishcst_aligned, thiso_aligned = xr.align(thishcst, thiso, join='inner')

        for icat in range(numcategories):
            # Compute category thresholds and probabilities for observations
            o_lo, o_hi = get_thresh(icat, quantiles, thiso_aligned, dims=['valid_time'])
            probo = 1. * np.logical_and(thiso_aligned > o_lo, thiso_aligned <= o_hi)

            if 'quantile' in probo:
                probo = probo.drop('quantile')

            l_probs_obs.append(probo.assign_coords({'category': icat}))

        thisobs = xr.concat(l_probs_obs, dim='category')

        # Now we can calculate the probabilistic (tercile categories) scores
        print('Now we can calculate the probabilistic (tercile categories) scores')
        thisroc = xr.Dataset()
        thisrps = xr.Dataset()
        thisrocss = xr.Dataset()
        thisbs = xr.Dataset()

        for var in thishcst_aligned.data_vars:
            thisroc[var] = xs.roc(thisobs[var], thishcst_aligned[var], dim='valid_time', bin_edges=np.linspace(0,1,101))

            thisrps[var] = xs.rps(thisobs[var], thishcst_aligned[var], dim='valid_time', category_edges=None, input_distributions='p')

            thisrocss[var] = (thisroc[var] - 0.5) / (1. - 0.5)

            bscat = list()
            for cat in thisobs[var].category:
                print(f'compute BS ({var}) for cat={cat.values}' )
                thisobscat = thisobs[var].sel(category=cat)
                thishcstcat = thishcst_aligned[var].sel(category=cat)
                bscat.append(xs.brier_score(thisobscat, thishcstcat, dim='valid_time'))

            thisbs[var] = xr.concat(bscat, dim='category')

        l_roc.append(thisroc)
        l_rps.append(thisrps)
        l_rocss.append(thisrocss)
        l_bs.append(thisbs)

    print('concat roc')
    roc = xr.concat(l_roc, dim='forecastMonth')
    print('concat rps')
    rps = xr.concat(l_rps, dim='forecastMonth')
    print('concat rocss')
    rocss = xr.concat(l_rocss, dim='forecastMonth')
    print('concat bs')
    bs = xr.concat(l_bs, dim='forecastMonth')

    print('writing to netcdf rps')
    rps.to_netcdf(f'{DATAOUT}/scores/{hcst_bname}.{aggr}.rps.nc')
    print('writing to netcdf bs')
    bs.to_netcdf(f'{DATAOUT}/scores/{hcst_bname}.{aggr}.bs.nc')
    print('writing to netcdf roc')
    roc.to_netcdf(f'{DATAOUT}/scores/{hcst_bname}.{aggr}.roc.nc')
    print('writing to netcdf rocss')
    rocss.to_netcdf(f'{DATAOUT}/scores/{hcst_bname}.{aggr}.rocss.nc')


## **4. Visualize verification plots**

## *4.1- CORRELATION*

In [ ]:
# Subset of plots to be produced
aggr  = '1m'
fcmonth = 2

# Common labels to be used in plot titles
VARNAMES = {
    't2m' : '2-metre temperature',
#     'sst' : 'sea-surface temperature',
#     'msl' : 'mean-sea-level pressure',
#     'tp'  : 'total precipitation'
}


CATNAMES=['lower tercile', 'middle tercile', 'upper tercile']

# Change titles font size
plt.rc('axes', titlesize=20)

In [ ]:
# PREPARE STRINGS for TITLES
origin_labels = {'institution': 'CMCC', 'name': 'SPSv3.5'}
tit_line1 = '{institution} {name}'.format(**origin_labels)
# tit_line2_base = rf'Start month: $\bf{calendar.month_abbr[config["start_month"]].upper()}$'
tit_line2_base = f'Start month: {calendar.month_abbr[config["start_month"]].upper()}'

if aggr=='1m':
    validmonth = config['start_month'] + (fcmonth-1)
    validmonth = validmonth if validmonth<=12 else validmonth-12
#     tit_line2 = tit_line2_base + rf' - Valid month: $\bf{calendar.month_abbr[validmonth].upper()}$'
    tit_line2 = tit_line2_base + f' - Valid month: {calendar.month_abbr[validmonth].upper()}'
elif aggr=='3m':
    validmonths = [vm if vm<=12 else vm-12 for vm in [config['start_month'] + (fcmonth-1) - shift for shift in range(3)]]
    validmonths = [calendar.month_abbr[vm][0] for vm in reversed(validmonths)]
#     tit_line2 = tit_line2_base + rf' - Valid months: $\bf{"".join(validmonths)}$'
    tit_line2 = tit_line2_base + f' - Valid months: {"".join(validmonths)}'
else:
    raise BaseException(f'Unexpected aggregation {aggr}')

In [ ]:
fcmonth

In [ ]:
# Read the data files
corr = xr.open_dataset(f'{DATAOUT}/scores/{hcst_bname}.{aggr}.corr.nc')
corr_pval = xr.open_dataset(f'{DATAOUT}/scores/{hcst_bname}.{aggr}.corr_pval.nc')
# RE-ARRANGE the DATASETS longitude values for plotting purposes
corr = corr.assign_coords(lon=(((corr.lon + 180) % 360) - 180)).sortby('lon')
corr_pval = corr_pval.assign_coords(lon=(((corr_pval.lon + 180) % 360) - 180)).sortby('lon')


thiscorr = corr.sel(forecastMonth=fcmonth)
thiscorrpval = corr_pval.sel(forecastMonth=fcmonth)

for var in thiscorr.data_vars:
    fig = plt.figure(figsize=(18,10))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.add_feature(cfeature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.add_feature(cfeature.COASTLINE, edgecolor='black', linewidth=2.)
    corrvalues = thiscorr[var].values
    corrpvalvalues = thiscorrpval[var].values

    if corrvalues.T.shape == (thiscorr[var].lat.size, thiscorr[var].lon.size):
        print('Data values matrices need to be transposed')
        corrvalues = corrvalues.T
        corrpvalvalues = corrpvalvalues.T
    elif corrvalues.shape == (thiscorr[var].lat.size, thiscorr[var].lon.size):
        pass
        # print('Data values matrices shapes are ok')
    else:
        raise BaseException(f'Unexpected data value matrix shape: {corrvalues.shape}' )


    plt.contourf(thiscorr[var].lon,thiscorr[var].lat,corrvalues,levels=np.linspace(-1.,1.,11),cmap='RdYlBu_r')
    cb = plt.colorbar(shrink=0.5)
    cb.ax.set_ylabel('Correlation',fontsize=12)
    origylim = ax.get_ylim()
    plt.contourf(thiscorrpval[var].lon,thiscorrpval[var].lat,corrpvalvalues,levels=[0.05,np.inf],hatches=['...',None],colors='none')
    # We need to ensure after running plt.contourf() the ylim hasn't changed
    if ax.get_ylim()!=origylim:
        ax.set_ylim(origylim)

    plt.title(tit_line1 + f' {VARNAMES[var]}' +' (stippling where significance below 95%)\n' + tit_line2,loc='left')


    plt.tight_layout()
    figname = f'{DATAOUT}/plots/stmonth{config["start_month"]:02d}/{hcst_bname}.{aggr}.fcmonth{fcmonth}.{var}.corr.png'
    plt.savefig(figname)

In [ ]:
import calendar
import ipywidgets as widgets

# Initial setup
hcst_bname="ecmwf_51_1993-2016_monthly_mean"
# config = {'start_month': 1}  # Modify as needed for your context
VARNAMES = {'t2m': '2-metre temperature'}
origin_labels = {'institution': 'CMCC', 'name': 'SPSv3.5'}
tit_line1 = '{institution} {name}'.format(**origin_labels)

# Load the datasets
def load_data(aggr):
    corr = xr.open_dataset(f'{DATAOUT}/scores/{hcst_bname}.{aggr}.corr.nc')
    corr_pval = xr.open_dataset(f'{DATAOUT}/scores/{hcst_bname}.{aggr}.corr_pval.nc')
    # Re-arrange longitude values for plotting purposes
    corr = corr.assign_coords(lon=(((corr.lon + 180) % 360) - 180)).sortby('lon')
    corr_pval = corr_pval.assign_coords(lon=(((corr_pval.lon + 180) % 360) - 180)).sortby('lon')
    return corr, corr_pval

# Define the plot function
def plot_corr(aggr, fcmonth):
    corr, corr_pval = load_data(aggr)

    # Prepare title line 2 based on aggregation
    tit_line2_base = f'Start month: {calendar.month_abbr[config["start_month"]].upper()}'
    if aggr == '1m':
        validmonth = config['start_month'] + (fcmonth - 1)
        validmonth = validmonth if validmonth <= 12 else validmonth - 12
        tit_line2 = tit_line2_base + f' - Valid month: {calendar.month_abbr[validmonth].upper()}'
    elif aggr == '3m':
        validmonths = [calendar.month_abbr[(config['start_month'] + (fcmonth - 1) - shift) % 12] for shift in range(3)]
        tit_line2 = tit_line2_base + f' - Valid months: {" ".join(reversed(validmonths))}'
    else:
        raise ValueError(f'Unexpected aggregation {aggr}')

    # Select data for the chosen forecast month
    thiscorr = corr.sel(forecastMonth=fcmonth)
    thiscorrpval = corr_pval.sel(forecastMonth=fcmonth)

    # Plot each variable in the dataset
    for var in thiscorr.data_vars:
        fig = plt.figure(figsize=(18, 10))
        ax = plt.axes(projection=ccrs.PlateCarree())
        ax.add_feature(cfeature.BORDERS, edgecolor='black', linewidth=0.5)
        ax.add_feature(cfeature.COASTLINE, edgecolor='black', linewidth=2.0)
        corrvalues = thiscorr[var].values
        corrpvalvalues = thiscorrpval[var].values

        if corrvalues.T.shape == (thiscorr[var].lat.size, thiscorr[var].lon.size):
            corrvalues = corrvalues.T
            corrpvalvalues = corrpvalvalues.T

        plt.contourf(thiscorr[var].lon, thiscorr[var].lat, corrvalues, levels=np.linspace(-1., 1., 11), cmap='RdYlBu_r')
        cb = plt.colorbar(shrink=0.5)
        cb.ax.set_ylabel('Correlation', fontsize=12)
        origylim = ax.get_ylim()
        plt.contourf(thiscorrpval[var].lon, thiscorrpval[var].lat, corrpvalvalues, levels=[0.05, np.inf], hatches=['...', None], colors='none')
        if ax.get_ylim() != origylim:
            ax.set_ylim(origylim)

        plt.title(f"{tit_line1} {VARNAMES[var]} (stippling where significance below 95%)\n{tit_line2}", loc='left')
        plt.tight_layout()
        plt.show()

# Create interactive widgets for `aggr` and `fcmonth`
aggr_dropdown = widgets.Dropdown(options=['1m', '3m'], value='1m', description='Aggregation:')
fcmonth_slider = widgets.IntSlider(value=3, min=2, max=4, step=1, description='Forecast Month:')

# Display the interactive plot with a single call
# out = widgets.interactive_output(plot_corr, {'aggr': aggr_dropdown, 'fcmonth': fcmonth_slider})
# display(aggr_dropdown, fcmonth_slider, out)
widgets.interactive(plot_corr, aggr=aggr_dropdown, fcmonth=fcmonth_slider)

## *4.2- RMSE*

In [ ]:
# Subset of plots to be produced
aggr  = '1m'
fcmonth = 4
# Common labels to be used in plot titles
VARNAMES = {
    't2m' : '2-metre temperature',
#     'sst' : 'sea-surface temperature',
#     'msl' : 'mean-sea-level pressure',
#     'tp'  : 'total precipitation'
}
CATNAMES=['lower tercile', 'middle tercile', 'upper tercile']
# Change titles font size
plt.rc('axes', titlesize=20)



# PREPARE STRINGS for TITLES
origin_labels = {'institution': 'CMCC', 'name': 'SPSv3.5'}
tit_line1 = 'RMSE-{institution} {name}'.format(**origin_labels)
# tit_line2_base = rf'Start month: $\bf{calendar.month_abbr[config["start_month"]].upper()}$'
tit_line2_base = f'RMSE-Start month: {calendar.month_abbr[config["start_month"]].upper()}'

if aggr=='1m':
    validmonth = config['start_month'] + (fcmonth-1)
    validmonth = validmonth if validmonth<=12 else validmonth-12
#     tit_line2 = tit_line2_base + rf' - Valid month: $\bf{calendar.month_abbr[validmonth].upper()}$'
    tit_line2 = tit_line2_base + f' - Valid month: {calendar.month_abbr[validmonth].upper()}'
elif aggr=='3m':
    validmonths = [vm if vm<=12 else vm-12 for vm in [config['start_month'] + (fcmonth-1) - shift for shift in range(3)]]
    validmonths = [calendar.month_abbr[vm][0] for vm in reversed(validmonths)]
#     tit_line2 = tit_line2_base + rf' - Valid months: $\bf{"".join(validmonths)}$'
    tit_line2 = tit_line2_base + f' - Valid months: {"".join(validmonths)}'
else:
    raise BaseException(f'Unexpected aggregation {aggr}')




# Read the data files
rmse = xr.open_dataset(f'{DATAOUT}/scores/{hcst_bname}.{aggr}.rmse.nc')
# RE-ARRANGE the DATASETS longitude values for plotting purposes
rmse = corr.assign_coords(lon=(((corr.lon + 180) % 360) - 180)).sortby('lon')


thisrmse = rmse.sel(forecastMonth=fcmonth)

for var in thisrmse.data_vars:
    fig = plt.figure(figsize=(18,10))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.add_feature(cfeature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.add_feature(cfeature.COASTLINE, edgecolor='black', linewidth=2.)
    rmsevalues = thisrmse[var].values

    if rmsevalues.T.shape == (thisrmse[var].lat.size, thisrmse[var].lon.size):
        print('Data values matrices need to be transposed')
        rmsevalues = rmsevalues.T
    elif rmsevalues.shape == (thisrmse[var].lat.size, thisrmse[var].lon.size):
        pass
        # print('Data values matrices shapes are ok')
    else:
        raise BaseException(f'Unexpected data value matrix shape: {rmsevalues.shape}' )


    plt.contourf(thisrmse[var].lon,thisrmse[var].lat,rmsevalues,levels=np.linspace(-1.,1.,11),cmap='RdYlBu_r')
    cb = plt.colorbar(shrink=0.5)
    cb.ax.set_ylabel('RMSE',fontsize=12)
    origylim = ax.get_ylim()
    # We need to ensure after running plt.contourf() the ylim hasn't changed
    if ax.get_ylim()!=origylim:
        ax.set_ylim(origylim)

    plt.title(tit_line1 + f' {VARNAMES[var]}' +' (stippling where significance below 95%)\n' + tit_line2,loc='left')


    plt.tight_layout()
    figname = f'{DATAOUT}/plots/stmonth{config["start_month"]:02d}/{hcst_bname}.{aggr}.fcmonth{fcmonth}.{var}.rmse.png'
    plt.savefig(figname)

In [ ]:
import calendar
import ipywidgets as widgets

# Initial setup
hcst_bname = "ecmwf_51_1993-2016_monthly_mean"
# config = {'start_month': 1}  # Modify as needed for your context
VARNAMES = {'t2m': '2-metre temperature'}
origin_labels = {'institution': 'CMCC', 'name': 'SPSv3.5'}
tit_line1 = 'RMSE---{institution} {name}'.format(**origin_labels)

# Load the datasets
def load_data(aggr):
    rmse = xr.open_dataset(f'{DATAOUT}/scores/{hcst_bname}.{aggr}.rmse.nc')
    # Re-arrange longitude values for plotting purposes
    rmse = rmse.assign_coords(lon=(((rmse.lon + 180) % 360) - 180)).sortby('lon')
    return rmse

# Define the plot function
def plot_rmse(aggr, fcmonth):
    rmse= load_data(aggr)

    # Prepare title line 2 based on aggregation
    tit_line2_base = f'Start month: {calendar.month_abbr[config["start_month"]].upper()}'
    if aggr == '1m':
        validmonth = config['start_month'] + (fcmonth - 1)
        validmonth = validmonth if validmonth <= 12 else validmonth - 12
        tit_line2 = tit_line2_base + f' - Valid month: {calendar.month_abbr[validmonth].upper()}'
    elif aggr == '3m':
        validmonths = [calendar.month_abbr[(config['start_month'] + (fcmonth - 1) - shift) % 12] for shift in range(3)]
        tit_line2 = tit_line2_base + f' - Valid months: {" ".join(reversed(validmonths))}'
    else:
        raise ValueError(f'Unexpected aggregation {aggr}')

    # Select data for the chosen forecast month
    thisrmse = rmse.sel(forecastMonth=fcmonth)

    # Plot each variable in the dataset
    for var in thisrmse.data_vars:
        fig = plt.figure(figsize=(18, 10))
        ax = plt.axes(projection=ccrs.PlateCarree())
        ax.add_feature(cfeature.BORDERS, edgecolor='black', linewidth=0.5)
        ax.add_feature(cfeature.COASTLINE, edgecolor='black', linewidth=2.0)
        rmsevalues = thisrmse[var].values

        if rmsevalues.T.shape == (thisrmse[var].lat.size, thisrmse[var].lon.size):
            rmsevalues = rmsevalues.T

        plt.contourf(thisrmse[var].lon, thisrmse[var].lat, rmsevalues, levels=np.linspace(np.nanmin(rmsevalues), np.nanmax(rmsevalues), 11), cmap='RdYlBu_r')
        cb = plt.colorbar(shrink=0.5)
        cb.ax.set_ylabel('RMSE', fontsize=12)
        origylim = ax.get_ylim()
        if ax.get_ylim() != origylim:
            ax.set_ylim(origylim)

        plt.title(f"{tit_line1} {VARNAMES[var]} (stippling where significance below 95%)\n{tit_line2}", loc='left')
        plt.tight_layout()
        plt.show()

# Create interactive widgets for `aggr` and `fcmonth`
aggr_dropdown = widgets.Dropdown(options=['1m', '3m'], value='1m', description='Aggregation:')
fcmonth_slider = widgets.IntSlider(value=10, min=2, max=4, step=1, description='Forecast Month:')

# Display the interactive plot with a single call
widgets.interactive(plot_rmse, aggr=aggr_dropdown, fcmonth=fcmonth_slider)

## *4-3 R2*

In [ ]:
# Subset of plots to be produced
aggr  = '1m'
fcmonth = 4
# Common labels to be used in plot titles
VARNAMES = {
    't2m' : '2-metre temperature',
#     'sst' : 'sea-surface temperature',
#     'msl' : 'mean-sea-level pressure',
#     'tp'  : 'total precipitation'
}
CATNAMES=['lower tercile', 'middle tercile', 'upper tercile']
# Change titles font size
plt.rc('axes', titlesize=20)



# PREPARE STRINGS for TITLES
origin_labels = {'institution': 'CMCC', 'name': 'SPSv3.5'}
tit_line1 = 'RMSE-{institution} {name}'.format(**origin_labels)
# tit_line2_base = rf'Start month: $\bf{calendar.month_abbr[config["start_month"]].upper()}$'
tit_line2_base = f'RMSE-Start month: {calendar.month_abbr[config["start_month"]].upper()}'

if aggr=='1m':
    validmonth = config['start_month'] + (fcmonth-1)
    validmonth = validmonth if validmonth<=12 else validmonth-12
#     tit_line2 = tit_line2_base + rf' - Valid month: $\bf{calendar.month_abbr[validmonth].upper()}$'
    tit_line2 = tit_line2_base + f' - Valid month: {calendar.month_abbr[validmonth].upper()}'
elif aggr=='3m':
    validmonths = [vm if vm<=12 else vm-12 for vm in [config['start_month'] + (fcmonth-1) - shift for shift in range(3)]]
    validmonths = [calendar.month_abbr[vm][0] for vm in reversed(validmonths)]
#     tit_line2 = tit_line2_base + rf' - Valid months: $\bf{"".join(validmonths)}$'
    tit_line2 = tit_line2_base + f' - Valid months: {"".join(validmonths)}'
else:
    raise BaseException(f'Unexpected aggregation {aggr}')




# Read the data files
rmse = xr.open_dataset(f'{DATAOUT}/scores/{hcst_bname}.{aggr}.rmse.nc')
# RE-ARRANGE the DATASETS longitude values for plotting purposes
rmse = corr.assign_coords(lon=(((corr.lon + 180) % 360) - 180)).sortby('lon')


thisrmse = rmse.sel(forecastMonth=fcmonth)

for var in thisrmse.data_vars:
    fig = plt.figure(figsize=(18,10))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.add_feature(cfeature.BORDERS, edgecolor='black', linewidth=0.5)
    ax.add_feature(cfeature.COASTLINE, edgecolor='black', linewidth=2.)
    rmsevalues = thisrmse[var].values

    if rmsevalues.T.shape == (thisrmse[var].lat.size, thisrmse[var].lon.size):
        print('Data values matrices need to be transposed')
        rmsevalues = rmsevalues.T
    elif rmsevalues.shape == (thisrmse[var].lat.size, thisrmse[var].lon.size):
        pass
        # print('Data values matrices shapes are ok')
    else:
        raise BaseException(f'Unexpected data value matrix shape: {rmsevalues.shape}' )


    plt.contourf(thisrmse[var].lon,thisrmse[var].lat,rmsevalues,levels=np.linspace(-1.,1.,11),cmap='RdYlBu_r')
    cb = plt.colorbar(shrink=0.5)
    cb.ax.set_ylabel('RMSE',fontsize=12)
    origylim = ax.get_ylim()
    # We need to ensure after running plt.contourf() the ylim hasn't changed
    if ax.get_ylim()!=origylim:
        ax.set_ylim(origylim)

    plt.title(tit_line1 + f' {VARNAMES[var]}' +' (stippling where significance below 95%)\n' + tit_line2,loc='left')


    plt.tight_layout()
    figname = f'{DATAOUT}/plots/stmonth{config["start_month"]:02d}/{hcst_bname}.{aggr}.fcmonth{fcmonth}.{var}.rmse.png'
    plt.savefig(figname)

In [ ]:
import calendar
import ipywidgets as widgets

# Initial setup
hcst_bname = "ecmwf_51_1993-2016_monthly_mean"
# config = {'start_month': 1}  # Modify as needed for your context
VARNAMES = {'t2m': '2-metre temperature'}
origin_labels = {'institution': 'CMCC', 'name': 'SPSv3.5'}
tit_line1 = 'R-2---{institution} {name}'.format(**origin_labels)

# Load the datasets
def load_data(aggr):
    rsquared = xr.open_dataset(f'{DATAOUT}/scores/{hcst_bname}.{aggr}.rsquared.nc')
    # Re-arrange longitude values for plotting purposes
    rsquared = rsquared.assign_coords(lon=(((rsquared.lon + 180) % 360) - 180)).sortby('lon')
    return rsquared

# Define the plot function
def plot_rsquared(aggr, fcmonth):
    rsquared= load_data(aggr)

    # Prepare title line 2 based on aggregation
    tit_line2_base = f'Start month: {calendar.month_abbr[config["start_month"]].upper()}'
    if aggr == '1m':
        validmonth = config['start_month'] + (fcmonth - 1)
        validmonth = validmonth if validmonth <= 12 else validmonth - 12
        tit_line2 = tit_line2_base + f' - Valid month: {calendar.month_abbr[validmonth].upper()}'
    elif aggr == '3m':
        validmonths = [calendar.month_abbr[(config['start_month'] + (fcmonth - 1) - shift) % 12] for shift in range(3)]
        tit_line2 = tit_line2_base + f' - Valid months: {" ".join(reversed(validmonths))}'
    else:
        raise ValueError(f'Unexpected aggregation {aggr}')

    # Select data for the chosen forecast month
    thisrsquared = rsquared.sel(forecastMonth=fcmonth)

    # Plot each variable in the dataset
    for var in thisrsquared.data_vars:
        fig = plt.figure(figsize=(18, 10))
        ax = plt.axes(projection=ccrs.PlateCarree())
        ax.add_feature(cfeature.BORDERS, edgecolor='black', linewidth=0.5)
        ax.add_feature(cfeature.COASTLINE, edgecolor='black', linewidth=2.0)
        rsquaredvalues = thisrsquared[var].values

        if rsquaredvalues.T.shape == (thisrsquared[var].lat.size, thisrsquared[var].lon.size):
            rsquaredvalues = rsquaredvalues.T

        plt.contourf(thisrsquared[var].lon, thisrsquared[var].lat, rsquaredvalues, levels=np.linspace(np.nanmin(rsquaredvalues), np.nanmax(rsquaredvalues), 30), cmap='RdYlBu_r')
        cb = plt.colorbar(shrink=0.5)
        cb.ax.set_ylabel('R-2', fontsize=12)
        origylim = ax.get_ylim()
        if ax.get_ylim() != origylim:
            ax.set_ylim(origylim)

        plt.title(f"{tit_line1} {VARNAMES[var]} (stippling where significance below 95%)\n{tit_line2}", loc='left')
        plt.tight_layout()
        plt.show()

# Create interactive widgets for `aggr` and `fcmonth`
aggr_dropdown = widgets.Dropdown(options=['1m', '3m'], value='1m', description='Aggregation:')
fcmonth_slider = widgets.IntSlider(value=10, min=2, max=4, step=1, description='Forecast Month:')

# Display the interactive plot with a single call
widgets.interactive(plot_rsquared, aggr=aggr_dropdown, fcmonth=fcmonth_slider)